# Installation

Install NVIDIA CUDA Toolkit: 
https://developer.nvidia.com/cuda-downloads?target_os=Windows&target_arch=x86_64&target_version=11&target_type=exe_local

-------------------------

```bash
conda create --name asr python=3.10 -y
conda activate asr
conda install pytorch==2.0.0 torchaudio==2.0.0 pytorch-cuda=11.8 -c pytorch -c nvidia
pip install git+https://github.com/m-bain/whisperx.git

```
Install ffmpeg (https://github.com/openai/whisper#setup)  
- Linux:
```bash
sudo apt update && sudo apt install ffmpeg
```
- Windows:
1) Install chocolatey -> follow instruction in (https://chocolatey.org/install), run commands in Windows PowerShell
2) ```bash
    choco install ffmpeg
    ```
3) Check if ffmpeg installed and what version:
```bash
    ffmpeg.exe -version
```
Install cudnn
```bash
conda install -c pypi cudnn -y
```

# Start service
The service will listen to port 8070
```bash
python segment-service.py
```

env for training
conda create --name test python=3.8  
!pip install datasets==1.18.3  
!pip install transformers==4.11.3  
!pip install huggingface_hub  
!pip install torchaudio     
!pip install librosa        
!pip install jiwer      
optuna     
matplotlib      
wandb 
pip install ipywidgets 
sudo apt install git-lfs


# Call the API

## Define the caller function

In [1]:
import requests,json,base64
import pandas as pd
import os
from glob import glob

## define a function for segmentation
def call_segment_service( service_address, 
                          audio_file_path,
                          reference_text,
                          language,
                          method
                        ):
    audio_file_base64_string = base64.b64encode( open(audio_file_path, 'rb').read()).decode('ASCII')
    response = requests.post( service_address,
                              data = json.dumps( {
                                  "audio_file_base64_string":audio_file_base64_string,
                                  "reference_text":reference_text,
                                  "language":language,
                                  "method":method
                              } ),
                              headers = {"Content-Type": "application/json"}
                            )
    return response.json()

## Segment Child Speech using WhisperX

## Get the corresponding reference text of each audio file

In [2]:
reading_material={
    "it":{
        "A":[
            "I draghi sono di solito giganteschi e pericolosi. Hanno squame, artigli e ali. Sembrano un incrocio tra un serpente, un pesce e un uccello. Alcuni draghi sputano fuoco. Altri hanno molte teste.",
            "I draghi depongono delle uova. Durante la schiusa, i piccoli draghi emergono dalle uova. I giovani draghi crescono molto lentamente. I draghi adulti possono vivere molto a lungo.",
            "I draghi esistono solo nei racconti. Ci sono racconti sui draghi in tantissimi paesi del mondo: in Cina, in Persia, in Grecia e in altri paesi europei. Benché nella maggior parte dei casi i draghi siano malvagi, mangiatori di uomini o guardiani di un tesoro, in Cina sono considerati come portafortuna."
        ],
        "B":[
            "Gli unicorni sono bianchi. Sono animali selvatici che assomigliano molto ai cavalli. Gli unicorni hanno un corno lungo e appuntito. Questo corno è contorto, un po' come una spirale. Il corno cresce in mezzo alla fronte.",
            "Gli unicorni vivono molto a lungo. Le leggende dicono che possono vivere anche secoli. I piccoli unicorni sono chiamati puledri. Le loro corna rimangono nascoste nella criniera fino all'età adulta.",
            "Gli unicorni sono un esempio perfetto di creature mitologiche. Si raccontavano storie di unicorni già a Roma e nella Grecia antica, ma anche in altri paesi europei. Questo animale mitologico è simbolo di purezza, perché il suo corno è un antidoto. È indomabile e molto difficile da catturare."
        ],
        "C":[
            "I nani sono in genere piccoli e carini. I nani assomigliano agli uomini. Hanno lunghe barbe bianche e portano un berretto a punta. Vivono spesso in gruppo nella stessa grotta. Sono in genere amichevoli.",
            "I nani lavorano sottoterra nelle miniere. Estraggono pietre preziose e oro dal sottosuolo. Sono incredibili gioiellieri. I loro gioielli sono splendidi e si vendono a caro prezzo.",
            "I nani compaiono spesso nelle fiabe. La maggior parte delle storie di nani proviene dalla Scandinavia, ma anche dalla Germania e da altri paesi europei. Sebbene i nani siano spesso di grande aiuto per gli esseri umani (e in questo caso sono chiamati spiriti buoni del focolare), ci sono tra loro anche dei guerrieri."
        ]
    },
    "fr":{
        "A":[
            "Les dragons sont souvent grands et dangereux. Les dragons ont des écailles, des griffes et des ailes. On dirait un croisement entre un serpent, un poisson et un oiseau. Certains dragons crachent du feu. D’autres ont plus d’une tête. ",
            "Les dragons pondent des oeufs. Lors de l'éclosion, les bébés dragons sortent des oeufs. Les jeunes dragons grandissent très lentement. Les dragons adultes peuvent alors vivre très vieux.",
            "Les dragons existent uniquement dans les contes. Il y a des histoires de dragons partout dans le monde, on en raconte en Chine, en Iran, en Grèce, et aussi dans d’autres pays européens. Bien que la plupart des dragons soient maléfiques, dévorent les humains ou protègent de précieux trésors, ils sont considérés comme des porte-bonheurs en Chine."
        ],
        "B":[
            "Une licorne est blanche. C'est un animal sauvage qui ressemble énormément à un cheval. Les licornes ont une corne longue et pointue. Leur corne est torsadée, un peu comme une spirale. Elle pousse au milieu de leur front.",
            "Les licornes vivent très longtemps. Des légendes racontent qu’elles pourraient même vivre plusieurs siècles. Les bébés licornes sont appelés poulains. Leur corne reste cachée dans leur crinière jusqu’à l’âge adulte.",
            "La licorne est l’exemple parfait de la créature mythique. On racontait déjà des histoires de licornes à Rome et en Grèce antique, mais aussi dans d’autres pays européens. Cet animal mythique est un signe de pureté, parce que sa corne est un antidote. Il est indomptable et très compliqué à capturer."
        ],
        "C":[
            "Les nains sont généralement petits et mignons. Les nains ressemblent à des hommes. Les nains ont de longues barbes blanches et portent un bonnet pointu. Ils vivent souvent à plusieurs dans la même grotte. Ils sont généralement amicaux.",
            "Les nains travaillent sous terre dans les mines. Ils extraient des pierres précieuses et de l’or du sous-sol. Ils sont d’incroyables bijoutiers. Leurs bijoux sont magnifiques et se vendent à prix d’or.",
            "Les nains apparaissent fréquemment dans les contes de fées. La plupart des histoires de nains proviennent de Scandinavie, mais aussi d'Allemagne ou d'autres pays européens. Quoique les nains soient parfois d’une aide considérable pour les humains, et sont alors appelés bons génies du foyer, il y a néanmoins des guerriers parmi eux."
        ]
    }
}

In [3]:
df = pd.read_csv("data/example/participants_extract.csv")
data_table = {"audio_path":[], "fname":[], "participant_id":[], "language":[],"form":[], "order":[], "reference_text":[]}
for path_name in glob("data/example/*/*.wav"):
    fname = os.path.basename( path_name )
    participant_id = int(fname.split("_")[0])
    lan = list(df[df["participant_id"] == participant_id]["language"])[0]
    form = list(df[df["participant_id"] == participant_id]["form"])[0]
    order = list(df[df["participant_id"] == participant_id]["order"])[0]

    if order == 0:
        print("skip ... likely invalid trial.")
        continue
    reference_text = reading_material[lan][form][order -1 ]

    data_table["audio_path"].append( path_name )
    data_table["fname"].append( fname )
    data_table["participant_id"].append( participant_id )
    data_table["language"].append( lan )
    data_table["form"].append( form )
    data_table["order"].append( order )
    data_table["reference_text"].append( reference_text )

data_table_df = pd.DataFrame( data_table )
data_table_df

skip ... likely invalid trial.
skip ... likely invalid trial.
skip ... likely invalid trial.
skip ... likely invalid trial.
skip ... likely invalid trial.
skip ... likely invalid trial.
skip ... likely invalid trial.
skip ... likely invalid trial.


,audio_path,fname,participant_id,language,form,order,reference_text
0,data/example\french\3101_edugame2023_823f70075...,3101_edugame2023_823f700759e54f17aae929dfce128...,3101,fr,B,2,Les licornes vivent très longtemps. Des légend...
1,data/example\french\3103_edugame2023_a8186e5c5...,3103_edugame2023_a8186e5c579c451692f69a0ddbee3...,3103,fr,B,1,Une licorne est blanche. C'est un animal sauva...
2,data/example\french\3104_edugame2023_0a936d5f3...,3104_edugame2023_0a936d5f3e25497fb00a4fcdbf72a...,3104,fr,C,1,Les nains sont généralement petits et mignons....
3,data/example\french\3106_edugame2023_8e887e74d...,3106_edugame2023_8e887e74d4414eecad4236774c62d...,3106,fr,A,3,Les dragons existent uniquement dans les conte...
4,data/example\french\3108_edugame2023_3c0a78208...,3108_edugame2023_3c0a78208d0c416c852225bccf336...,3108,fr,C,2,Les nains travaillent sous terre dans les mine...
5,data/example\french\3109_edugame2023_00b0af188...,3109_edugame2023_00b0af1883a14ceeb73a49704187e...,3109,fr,C,1,Les nains sont généralement petits et mignons....
6,data/example\french\3110_edugame2023_01e806baa...,3110_edugame2023_01e806baa12f4df891326e4acade6...,3110,fr,A,1,Les dragons sont souvent grands et dangereux. ...
7,data/example\french\3111_edugame2023_090bbbdda...,3111_edugame2023_090bbbdda6cc499d8f12cbec6256e...,3111,fr,B,3,La licorne est l’exemple parfait de la créatur...
8,data/example\french\3116_edugame2023_343062cee...,3116_edugame2023_343062cee0ce4572b692dc1108d03...,3116,fr,C,2,Les nains travaillent sous terre dans les mine...
9,data/example\french\3117_edugame2023_a4257b0c1...,3117_edugame2023_a4257b0c11684b929f531dc73b0c5...,3117,fr,A,3,Les dragons existent uniquement dans les conte...


## Transcribe an audio, and align with the reference text

### check the french

In [26]:
participant_id  = 3104
data_table_df.loc[ data_table_df["participant_id"] == participant_id ]

,audio_path,fname,participant_id,language,form,order,reference_text
2,data/example\french\3104_edugame2023_0a936d5f3...,3104_edugame2023_0a936d5f3e25497fb00a4fcdbf72a...,3104,fr,C,1,Les nains sont généralement petits et mignons....


In [27]:
audio_path = list(data_table_df.loc[ data_table_df["participant_id"] == participant_id ]["audio_path"])[0]
reference_text = list(data_table_df.loc[ data_table_df["participant_id"] == participant_id ]["reference_text"])[0]
language = list(data_table_df.loc[ data_table_df["participant_id"] == participant_id ]["language"])[0]
audio_path, reference_text, language

('data/example\\french\\3104_edugame2023_0a936d5f3e25497fb00a4fcdbf72aac6_9cbecd83e198456db059ff879bc134a8.wav',
 'Les nains sont généralement petits et mignons. Les nains ressemblent à des hommes. Les nains ont de longues barbes blanches et portent un bonnet pointu. Ils vivent souvent à plusieurs dans la même grotte. Ils sont généralement amicaux.',
 'fr')

In [29]:
call_segment_service( 
            service_address = "http://localhost:8070/segment",  ### This is the service address, default on 8070 port.
            audio_file_path = audio_path,  ### path to the audio file
            reference_text = reference_text,
            language = language,   ### "it" for "italian", "fr" for "french", "de" for "german"
            method = "ibm-watson"  ### The name of the model, can be either "whisperx" or "ibm-watson"
)

{'transcription': {'start': 3.3,
  'end': 89.5,
  'text': 'les généralement petits et mignons les nains et semblent à des de longues bras blancs et portent des bonnet pointure et ils vivent souvent à plusieurs dans la main ils sont généralement amis non travaillent sous la terre dans les mines et des pierres précieuses et de lors du sous son et ils sont incapables bes et les bijoux sont magnifiques et sont',
  'words': [{'word': 'les', 'start': 3.3, 'end': 4.24, 'score': 0.59},
   {'word': 'généralement', 'start': 6.1, 'end': 7.64, 'score': 0.69},
   {'word': 'petits', 'start': 7.96, 'end': 8.52, 'score': 0.78},
   {'word': 'et', 'start': 8.62, 'end': 8.76, 'score': 0.8},
   {'word': 'mignons', 'start': 8.94, 'end': 9.48, 'score': 0.4},
   {'word': 'les', 'start': 10.2, 'end': 10.42, 'score': 0.71},
   {'word': 'nains', 'start': 10.7, 'end': 11.1, 'score': 0.31},
   {'word': 'et', 'start': 11.86, 'end': 12, 'score': 0.5},
   {'word': 'semblent', 'start': 12.28, 'end': 12.82, 'score': 0

### Check the Italian

In [18]:
participant_id  = 102
data_table_df.loc[ data_table_df["participant_id"] == participant_id ]

audio_path = list(data_table_df.loc[ data_table_df["participant_id"] == participant_id ]["audio_path"])[0]
reference_text = list(data_table_df.loc[ data_table_df["participant_id"] == participant_id ]["reference_text"])[0]
language = list(data_table_df.loc[ data_table_df["participant_id"] == participant_id ]["language"])[0]
audio_path, reference_text, language

('data/example\\italian\\102_edugame2023_67b3311a82a84f49a9c76d23fb385084_2e5d8a11f48f4c3eae6dff081ae5059f.wav',
 'I nani compaiono spesso nelle fiabe. La maggior parte delle storie di nani proviene dalla Scandinavia, ma anche dalla Germania e da altri paesi europei. Sebbene i nani siano spesso di grande aiuto per gli esseri umani (e in questo caso sono chiamati spiriti buoni del focolare), ci sono tra loro anche dei guerrieri.',
 'it')

In [19]:
call_segment_service( 
            service_address = "http://localhost:8070/segment",  ### This is the service address, default on 8070 port.
            audio_file_path = audio_path,  ### path to the audio file
            reference_text = reference_text,
            language = language,   ### "it" for "italian", "fr" for "frence", "de" for "german"
            method = "whisperx"  ### The name of the model, can be either "whisperx" or "ibm-watson"
)

{'transcription': {'start': 0.149,
  'end': 46.812,
  'text': ' I nani capiscono spesso nelle viabbe la maggior parte delle storie di nani proviene dalla Scandinavia ma anche dalla Germania e  da altri paesi europei, sebbene i nani siano spesso di grande aiuto per gli',
  'words': [{'word': 'I', 'start': 0.149, 'end': 0.429, 'score': 0.173},
   {'word': 'nani', 'start': 2.71, 'end': 3.01, 'score': 0.886},
   {'word': 'capiscono', 'start': 3.07, 'end': 3.911, 'score': 0.884},
   {'word': 'spesso', 'start': 4.071, 'end': 4.891, 'score': 0.939},
   {'word': 'nelle', 'start': 5.352, 'end': 5.812, 'score': 0.919},
   {'word': 'viabbe', 'start': 5.952, 'end': 6.532, 'score': 0.86},
   {'word': 'la', 'start': 6.612, 'end': 6.852, 'score': 0.821},
   {'word': 'maggior', 'start': 8.173, 'end': 8.893, 'score': 0.91},
   {'word': 'parte', 'start': 9.053, 'end': 9.554, 'score': 0.976},
   {'word': 'delle', 'start': 9.674, 'end': 10.254, 'score': 0.962},
   {'word': 'storie', 'start': 11.254, 'end'